In [1]:
from keras import *
from keras.layers import Conv2D, MaxPooling2D, Input
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en')

from collections import defaultdict as dd

Using TensorFlow backend.
c:\users\kevin\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
def InceptionModule(x):
    tower_1 = Conv2D(64, (x.shape[1], 1), padding='valid', activation='relu')(x)
    tower_1 = Conv2D(64, (x.shape[1], 3), padding='valid', activation='relu')(tower_1)

    tower_2 = Conv2D(64, (x.shape[1], 1), padding='valid', activation='relu')(x)
    tower_2 = Conv2D(64, (x.shape[1], 5), padding='valid', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((x.shape[1], 3), strides=(1, 1), padding='valid')(x)
    tower_3 = Conv2D(64, (x.shape[1], 1), padding='valid', activation='relu')(tower_3)
    
    tower_4 = Conv2D(64, (x.shape[1], 1), padding='valid', activation='relu')(x)

    return keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis=1)

In [ ]:
w2v = Word2Vec.load("./models/gutenberg_w2v.model")
embedding = w2v.wv.get_keras_embedding()

In [ ]:
inputs = Input(shape=(3, 96, 96))
x = embedding(inputs)
x = InceptionModule(x)
x = InceptionModule(x)
x = Flatten()(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(input=inputs, output=outputs)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])